In [2]:
import import_ipynb
import preprocess_model
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dense, Embedding, Flatten, Concatenate, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

importing Jupyter notebook from preprocess_model.ipynb
Shape: (1316, 26)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1316 entries, 0 to 1315
Data columns (total 26 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Total Population                            1316 non-null   float64
 1   Female Population                           1316 non-null   float64
 2   Male Population                             1316 non-null   float64
 3   Birth Rate                                  1269 non-null   float64
 4   Death Rate                                  1269 non-null   float64
 5   Compulsory Education Dur.                   1041 non-null   float64
 6   Employment in Industry(%)                   1267 non-null   float64
 7   Employment in Agriculture(%)                1267 non-null   float64
 8   Female Employment in Agriculture(%)         1267 non-null   float64
 9   Female Emplo

In [3]:
df = pd.read_csv('./data/new_data_asia.csv')

In [4]:
# Select relevant features and target
features = ['School enrollment, primary', 'School enrollment, tertiary', 'Primary completion rate', 'Year']
Country = ['Country']
target = 'Literacy rate'

df.dropna(subset=[target], inplace=True)


In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(df[features + Country], df[target], test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Shapes after splitting:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:", X_val.shape, "y_val:", y_val.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)


Shapes after splitting:
X_train: (160, 5) y_train: (160,)
X_val: (53, 5) y_val: (53,)
X_test: (54, 5) y_test: (54,)


In [7]:
# Split data into training, validation, and testing sets
scaler = StandardScaler()
X_train[features] = scaler.fit_transform(X_train[features])
X_val[features] = scaler.transform(X_val[features])
X_test[features] = scaler.transform(X_test[features])


In [ ]:
label_encoders = {}
for cat_feature in categorical_features:
    le = LabelEncoder()
    X_train[cat_feature] = le.fit_transform(X_train[cat_feature])
    X_val[cat_feature] = le.transform(X_val[cat_feature])
    X_test[cat_feature] = le.transform(X_test[cat_feature])
    label_encoders[cat_feature] = le

# Define the learning rate
learning_rate = 0.0001  

In [59]:
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(2)  # 2 đầu ra cho 2 cột cần dự đoán
])

model.compile(optimizer='adam', loss='mean_squared_error')

In [60]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)


Epoch 1/50
33/33 [==============================] - 1s 9ms/step - loss: nan - val_loss: nan
Epoch 2/50
33/33 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 3/50
33/33 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 4/50
33/33 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 5/50
33/33 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 6/50
33/33 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 7/50
33/33 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 8/50
33/33 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 9/50
33/33 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 10/50
33/33 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 11/50
33/33 [==============================] - 0s 2ms/step - loss: nan - 

In [62]:
predicted_values_scaled = model.predict(X_test)


9/9 [==============================] - 0s 997us/step


In [63]:
predicted_values = scaler_targets.inverse_transform(predicted_values_scaled)
